# SonoYolo: Finetuning YOLOv11n

In [1]:
from ultralytics import YOLO

In [2]:
model = YOLO("yolo11n.pt")

In [12]:
#This cell prepares the data for training
import os
import cv2
import pandas as pd
from tqdm import tqdm

# --- CONFIGURATION ---
video_dir = "/shared_data/p_vidalr/iraj/EchoNet-Dynamic/Videos"
output_root = "echonet_yolo"
original_size = 112         # Original size of frames (EchoNet-Dynamic Dataset
padded_size = 128           # Padded file size for YOLOv11n
pad = (padded_size - original_size) // 2  # 8 pixels on each side

vol_csv = "/shared_data/p_vidalr/iraj/EchoNet-Dynamic/VolumeTracings.csv" #Volume Tracings File in Stanford Echonet-Dynamic Dataset File
filelist_csv = "/shared_data/p_vidalr/iraj/EchoNet-Dynamic/FileList.csv" #We are using the splits that the Stanford EchoNet-Dynamic Model used for training, testing and validation

# --- Create output folders ---
os.makedirs(output_root, exist_ok=True)
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(output_root, f"images/{split}"), exist_ok=True)
    os.makedirs(os.path.join(output_root, f"labels/{split}"), exist_ok=True)

# --- Load annotations (VolumeTracings) ---
df = pd.read_csv(vol_csv)
if "FileName" in df.columns:
    tracings_file_col = "FileName"
elif "Filename" in df.columns:
    tracings_file_col = "Filename"
else:
    raise ValueError("VolumeTracings.csv must contain 'FileName' or 'Filename' column.")

df[tracings_file_col] = df[tracings_file_col].astype(str).str.strip()

# --- Load EchoNet-style split list (FileList.csv) ---
fl = pd.read_csv(filelist_csv)


if "FileName" in fl.columns:
    filelist_file_col = "FileName"
elif "Filename" in fl.columns:
    filelist_file_col = "Filename"
else:
    
    filelist_file_col = fl.columns[0]

if "Split" in fl.columns:
    split_col = "Split"
elif "split" in fl.columns:
    split_col = "split"
else:
    raise ValueError("FileList.csv must contain 'Split' (or 'split') column with values TRAIN/VAL/TEST.")

# Normalize filenames (as given, these are basenames without extension)
fl[filelist_file_col] = fl[filelist_file_col].astype(str).str.strip()

# Normalize split labels from TRAIN/VAL/TEST -> train/val/test
fl[split_col] = (
    fl[split_col]
    .astype(str).str.strip().str.upper()
    .map({"TRAIN": "train", "VAL": "val", "TEST": "test"})
)


fl["_with_ext"] = fl[filelist_file_col] + ".avi"

# dataset splits from FileList.csv
splits = {
    'train': sorted(fl.loc[fl[split_col] == 'train', "_with_ext"].unique().tolist()),
    'val':   sorted(fl.loc[fl[split_col] == 'val',   "_with_ext"].unique().tolist()),
    'test':  sorted(fl.loc[fl[split_col] == 'test',  "_with_ext"].unique().tolist()),
}

print({k: len(v) for k, v in splits.items()})

# --- Pre-index tracings per video for speed ---
grouped = {k: g for k, g in df.groupby(tracings_file_col)}

# --- MAIN LOOP ---
for split, video_list in splits.items():
    img_dir = os.path.join(output_root, f"images/{split}")
    lbl_dir = os.path.join(output_root, f"labels/{split}")

    for video_name in tqdm(video_list, desc=f"Processing {split}"):
        # Skip if no tracings (paranoia; we already filtered)
        if video_name not in grouped:
            continue

        video_path = os.path.join(video_dir, video_name)
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Failed to open {video_name}")
            continue

        # Get annotated frames for this video
        frames_df = grouped[video_name]
        unique_frames = frames_df['Frame'].dropna().astype(int).unique()
        if len(unique_frames) < 2:
            cap.release()
            continue

        # Choose first 2 annotated frames (ED/ES typically present)
        selected_frames = sorted(unique_frames)[:2]

        for frame_num in selected_frames:
            frame_rows = frames_df[frames_df['Frame'].astype(int) == frame_num]
            if frame_rows.empty:
                continue

            # Get bbox corners from tracings
            x_vals = pd.concat([frame_rows['X1'], frame_rows['X2']])
            y_vals = pd.concat([frame_rows['Y1'], frame_rows['Y2']])

            xmin, xmax = int(x_vals.min()), int(x_vals.max())
            ymin, ymax = int(y_vals.min()), int(y_vals.max())

            # Read frame (EchoNet videos are already 112x112)
            cap.set(cv2.CAP_PROP_POS_FRAMES, int(frame_num))
            ret, frame = cap.read()
            if not ret:
                print(f"Could not read frame {frame_num} in {video_name}")
                continue

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # === Pad to 128×128 ===
            frame_padded = cv2.copyMakeBorder(
                frame_rgb, pad, pad, pad, pad,
                borderType=cv2.BORDER_CONSTANT,
                value=(0, 0, 0)
            )

            # === Adjust bounding box (shift by pad; normalize by 128) ===
            x_center = ((xmin + xmax) / 2 + pad) / padded_size
            y_center = ((ymin + ymax) / 2 + pad) / padded_size
            width = (xmax - xmin) / padded_size
            height = (ymax - ymin) / padded_size

            # === Save image and label ===
            base_name = f"{video_name.replace('.avi', '')}_frame{frame_num}"
            image_path = os.path.join(img_dir, f"{base_name}.jpg")
            label_path = os.path.join(lbl_dir, f"{base_name}.txt")

            cv2.imwrite(image_path, frame_padded)
            with open(label_path, "w") as f:
                f.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

        cap.release()

print("Done.")


{'train': 7465, 'val': 1288, 'test': 1277}


Processing test: 100%|██████████| 1277/1277 [00:13<00:00, 93.53it/s] 

Done.


In [13]:
from ultralytics import YOLO
from ultralytics import YOLO
import os

# Edit based on your needs
DATA_YAML = "/home/iraj/echonet_yolo/echonet_yolo.yaml"
TRAIN_EPOCHS = 40
IMG_SIZE = 128
BATCH_SIZE = 16
TRAIN_WEIGHTS = "yolo11n.pt"  # pretrained weights to start from


# Training

print("Loading pretrained YOLOv11n model")
model = YOLO(TRAIN_WEIGHTS)

print(f"Training model on dataset: {DATA_YAML}")
model.train(
    data=DATA_YAML,
    epochs=TRAIN_EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    pretrained=True  # use pretrained weights and adapt head
)

Loading pretrained YOLOv8n model...
Training model on dataset: /home/iraj/echonet_yolo/echonet_yolo.yaml
Ultralytics 8.3.169 🚀 Python-3.8.20 torch-2.4.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24149MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/iraj/echonet_yolo/echonet_yolo.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=128, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opse

train: Scanning /home/iraj/echonet_yolo/labels/train... 14920 images, 0 backgrounds, 0 corrupt: 100%|██████████| 14920/14920 [00:05<00:00, 2705.35it/s]


train: New cache created: /home/iraj/echonet_yolo/labels/train.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4.3±5.2 MB/s, size: 5.6 KB)


val: Scanning /home/iraj/echonet_yolo/labels/val... 2576 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2576/2576 [00:01<00:00, 2478.71it/s]

val: New cache created: /home/iraj/echonet_yolo/labels/val.cache


...

      40/40      0.23G     0.8825     0.3252     0.9389          8        128: 100%|██████████| 933/933 [00:53<00:00, 17.32it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 81/81 [00:05<00:00, 14.75it/s]

                   all       2576       2576      0.998      0.998      0.995      0.779



40 epochs completed in 0.670 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 5.4MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics 8.3.169 🚀 Python-3.8.20 torch-2.4.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24149MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 81/81 [00:06<00:00, 13.18it/s]


                   all       2576       2576      0.998      0.998      0.995      0.779
Speed: 0.0ms preprocess, 0.3ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train4


In [14]:
best_weights_path = "/home/iraj/runs/detect/train4/weights/best.pt" #after training you should get best weights
model = YOLO(best_weights_path)

# Run inference on a single image or folder of images
results = model.predict(
    source="/home/iraj/echonet_yolo/images/test",  # can be image, folder, or video
    imgsz= 128,  # keep consistent with training image size
    conf=0.65,  # confidence threshold
    save=True,  # save the results to 'runs/detect/exp' folder by default
    save_txt=True  # optionally save prediction labels in txt files
)


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/2552 /home/iraj/echonet_yolo/images/test/0X100CF05D141FF143_frame132.jpg: 128x128 1 left_ventricle, 6.8ms
image 2/2552 /home/iraj/echonet_yolo/images/test/0X100CF05D141FF143_frame148.jpg: 128x128 1 left_ventricle, 6.9ms
image 3/2552 /home/iraj/echonet_yolo/images/test/0X1012703CDC1436FE_frame154.jpg: 128x128 1 left_ventricle, 6.8ms
image 4/2552 /home/iraj/echonet_yolo/images/test/0X1012703CDC1436FE_frame172.jpg: 128x128 1 left_ventricle, 6.8ms
